In [1]:
# US Bonds - BND, Bloomberg Barclays U.S. Aggregate Float Adjusted Index:
#            Using FTSE US Broad Investment-Grade Bond Index (USBIG®) instead

In [2]:
import numpy as np
import pandas as pd
from PyPDF2 import PdfFileReader
import re
import datetime
import requests
import os

In [3]:
bndURL = "https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20191031&IsManual=true"

In [4]:
%%time
year = ''
month = ''
day = ''

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [5]:
cur_year = datetime.datetime.now().year
cur_year

2020

In [6]:
error_url = 'https://www.ftserussell.com/products/indices/home/errorview?aspxerrorpath=/Analytics/FactSheets/Home/DownloadSingleIssueByDate'
error_url2 = 'https://research.ftserussell.com/Analytics/FactSheets/Home/ErrorPage?ErrorMessage=There%20has%20been%20a%20problem%20downloading%20this%20file'

In [7]:
quarterly_links = []

In [8]:
%%time
# Quarterly
for y in range(2018,cur_year+1):
    year = str(y)
    for m in range(3,13,3):
        if m < 10:
            month = '0'+ str(m)
        else:
            month = str(m)
        for d in range(28,32):
            day = str(d)
            date = year + month + day
            status = 'true'
            url = "https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=USBIG&IssueDate="+date+"&IsManual="+status
            response = requests.get(url)
            if response.url == error_url or response.url == error_url2:
                status = 'false'
                url = "https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=USBIG&IssueDate="+date+"&IsManual="+status
                response = requests.get(url)
                if response.url == error_url or response.url == error_url2:
                    print(date + " Failed")
            elif response.url != error_url and response.url != error_url2:
                print(url)
                quarterly_links.append(url)
                break

20180328 Failed
20180329 Failed
20180330 Failed
20180331 Failed
20180628 Failed
20180629 Failed
20180630 Failed
20180631 Failed
20180928 Failed
20180929 Failed
20180930 Failed
20180931 Failed
20181228 Failed
20181229 Failed
20181230 Failed
20181231 Failed
20190328 Failed
20190329 Failed
20190330 Failed
20190331 Failed
20190628 Failed
20190629 Failed
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190630&IsManual=true
20190928 Failed
20190929 Failed
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190930&IsManual=true
20191228 Failed
20191229 Failed
20191230 Failed
https://research.ftserussell.com/Analytics/FactSheets/Home/DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20191231&IsManual=true
20200328 Failed
20200329 Failed
20200330 Failed
20200331 Failed
20200628 Failed
20200629 Failed
20200630 Failed
20200631 Failed
20200928 Failed
20200929 Failed
202009

In [9]:
# Downloading files
for url in quarterly_links: 
    file_name = url.split('/')[-1]    
    print("Downloading file:%s"%file_name) 
    r = requests.get(url, stream = True) 
    with open(file_name, 'wb') as f: 
        for chunk in r.iter_content(chunk_size = 1024*1024): 
            if chunk: 
                f.write(chunk) 
    print("%s downloaded!\n"%file_name)
print("All reports downloaded!")

DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190630&IsManual=true downloaded!

DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190930&IsManual=true downloaded!

DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20191231&IsManual=true downloaded!

All reports downloaded!


In [10]:
file_names = []
for url in quarterly_links:
    file = url.split('/')[-1]
    file_names.append(file)

In [11]:
for file in file_names:
    print(file)
    os.rename(file,file+'.pdf')

DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190630&IsManual=true
DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190930&IsManual=true
DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20191231&IsManual=true


In [12]:
file_names1 = []
for file in file_names:
    file_names1.append(file+'.pdf')

In [13]:
bndDF = pd.DataFrame(columns=['File','MC'])
for x in range(len(file_names1)):
    pdf = PdfFileReader(file_names1[x])
    pdf_page = pdf.getPage(1)
    pdf_text = pdf_page.extractText()
    pdf_text = pdf_text.replace('\n',' ')
    pdf_text = pdf_text.replace('  ',' ')
    pdf_text = pdf_text.replace('  ',' ')
    mc = re.findall('Totals \d+ \d+,\d+,\d+', pdf_text, re.IGNORECASE)
    if mc == []:
        mc = 'n/a'
        row = [file_names1[x],mc]
        bndDF.loc[x] = row
        continue
    else:
        mc_1 = mc[0].replace(',','')
        mc_2 = mc_1.replace('\n','')
        mc_3 = mc_2.replace('Totals','')
        mc = [int(s) for s in mc_3.split() if s.isdigit()][1]
        row = [file_names1[x],mc]
        bndDF.loc[x] = row

In [14]:
file_names1[0]

'DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190630&IsManual=true.pdf'

In [15]:
pdf = PdfFileReader(file_names1[0])
pdf_page = pdf.getPage(0)
pdf_text = pdf_page.extractText()

In [16]:
pdf_text

'ftserussell.comSource: FTSE Russell as of June 30, 2019. Past performance is no guarantee of future results. Returns shown prior to October 1, 1985 reflect hypothetical historical performance.Please see the end for important legal disclosures.FTSE US Broad Investment-Grade Bond Index (USBIG®)| 01FTSE Russell Factsheet | June 30, 2019FTSE US Broad Investment-Grade Bond Index (USBIG®)Broad | US DollarThe FTSE US Broad Investment-Grade Bond Index (USBIG) measures the performance of US Dollar-denominated bonds issued in the US investment-grade bond market. Introduced in 1985, the index covers US Treasury, government sponsored, collateralized, and corporate debt providing a reliable representation of the US investment-grade bond market. Sub-indexes are available in any combination of asset class, maturity, and rating.INDEX PROFILEDescription# of  IssuesPar Amount*Market Value*MarketWeight (%)AverageCoupon (%)Average Life(Years)Yield toMaturity (%)Effective DurationOAS (bps)USBIG7,84120,693

In [17]:
re.findall('\d+100.00',pdf_text)

['82100.00']

In [18]:
bndDF

,File,MC
0,DownloadSingleIssueByDate?IssueName=USBIG&Issu...,n/a
1,DownloadSingleIssueByDate?IssueName=USBIG&Issu...,n/a
2,DownloadSingleIssueByDate?IssueName=USBIG&Issu...,n/a


In [19]:
# Deleting files
for file in file_names1:
    print("Deleting file:%s"%file) 
    os.remove(file)
    print("%s deleted!\n"%file)
print("All reports deleted!")

Deleting file:DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190630&IsManual=true.pdf
DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190630&IsManual=true.pdf deleted!

Deleting file:DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190930&IsManual=true.pdf
DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20190930&IsManual=true.pdf deleted!

Deleting file:DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20191231&IsManual=true.pdf
DownloadSingleIssueByDate?IssueName=USBIG&IssueDate=20191231&IsManual=true.pdf deleted!

All reports deleted!
